In [ ]:
#imports
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from pymongo import MongoClient

# mongo configuration
myclient = MongoClient("mongodb://localhost:27017/")
mydb = myclient["uefa"]
mycol = mydb["players-ok"]

#player class
class Player(object):
            def __init__(self, _id, name, team, goals, saves, offsides,
                         minutesPlayed, goalsConceded, played, yellowCards,redCards):
                self.id = _id
                self.name = name
                self.team = team
                self.goals = goals
                self.saves = saves
                self.offsides = offsides
                self.minutesPlayed = minutesPlayed
                self.goalsConceded = goalsConceded
                self.played = played
                self.yellowCards = yellowCards
                self.redCards=redCards
                    
driver = webdriver.Chrome()

#go to the main page
clubs = "https://www.uefa.com/uefachampionsleague/clubs/"
driver.get(clubs)

#save number of links to teams
nTeams = len(driver.find_elements_by_css_selector("a.team-wrap"))

#first loop: teams
for currentTeam in range(nTeams):
    #save links to teams
    cssSelector = "a.team-wrap"
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, cssSelector)))
    except:
        time.sleep(5)
    teams = driver.find_elements_by_css_selector(cssSelector)
    
    #save href attribute and go to squad page
    teamLink = str(teams[currentTeam].get_attribute('href')) + "/squad"
    driver.get(teamLink)
    
    #save team name
    team = driver.find_element_by_css_selector("h1.team-name").text
    
    #save number of players
    nPlayers = len(driver.find_elements_by_css_selector("a.player-name"))
    
    #second loop: players
    for currentPlayer in range(nPlayers):
        cssSelector = "a.player-name"
        player = driver.find_elements_by_css_selector(cssSelector)
        time.sleep(2)
        driver.execute_script("arguments[0].click();", player[currentPlayer])
        time.sleep(2)

        total = {}
        statisticsListBlock = driver.find_elements_by_css_selector("div.player--statistics--list")
        
        # save player's id and name
        url = driver.current_url
        x = url.split('/')
        data = x[6].split('--')
        player_id = data[0]
        name = driver.find_element_by_css_selector("h1.player-header_name").text     
        
        # if there are statistics, save it
        if(len(statisticsListBlock) == 1):
            fields = driver.find_elements_by_css_selector("div.player--statistics--list .field")            
            
            # get data from fields
            for field in fields:
                label = field.find_element_by_class_name("statistics--list--label").text
                data = field.find_element_by_class_name("statistics--list--data").text
                if(label == "CARDS"):
                    cards = data.split(' ')
                    total.update({"YELLOW CARDS" : cards[0]})        
                    total.update({"RED CARDS" : cards[1]})
                else:
                    total.update({label: data})

            # save data in a player object
            try:
                player = Player(player_id, name, team, total["GOALS"], total["SAVES"], total["OFFSIDES"],
                                total["MINUTES PLAYED"], total["GOALS CONCEDED"], total["PLAYED"], total["YELLOW CARDS"], total["RED CARDS"])
            except KeyError:
                player = Player(player_id, name, team, total.get("GOALS"), total.get("SAVES"), total.get("OFFSIDES"),
                                total.get("MINUTES PLAYED"), total.get("GOALS CONCEDED"), total.get("PLAYED"), total.get("YELLOW CARDS"), total.get("RED CARDS"))            
        # if there are no statistics... just save 0
        else:
            player = Player(player_id, name, team, "0", "0", "0", "0", "0", "0", "0", "0")        
        
        # add data to a dict
        playerData = {
            "player id" : player.id,
            "name" : player.name,
            "team" : player.team,
            "goals" : player.goals,
            "saves" : player.saves,
            "offsides" : player.offsides,
            "minutes played" : player.minutesPlayed,
            "goals conceeded" : player.goalsConceded,
            "played" : player.played,
            "yellow cards" : player.yellowCards,
            "red cards" : player.redCards
        }
        
        #insert player data into mongodb
        x = mycol.insert_one(playerData)
        
        #finish player and go back to team page
        driver.get(teamLink)

    #finish team and go back to main page
    driver.get(clubs)

driver.close()